# Introduction to Decorators
- Specify management or augmentation code for functions and classes
    - Rebinding function and class to other callables at the end of **def** and **class**
- Decorators themselves take the form of callable objects
- Do not need new-style class
- A kind of runtime declaration

# Calls and Instance
- Call proxies (Function Decorators)
    - Intercept later function calls and process them as needed
- Interface proxies (Class Decorators)
    - Intercept later instnace creation calls and process them as needed
    
# Function Decorator VS Class Decorator
- Function decorator
    - Only a specific function
- Class decorator
    - All instance creation calls
    
    
# Why Decorators?
- Make intent clearer
- Minimize augmentation code redundancy
- Help ensure correct API usage
- Explicity syntax
- Mainability
    - Not necessary to add extra code at every calll to the class or function
    
## Drawbacks
- Alter type of decorated objects
- Extra call (costs)
    

# Basics

## Function Decorators

### Usage

```python
@decorator
def fun(arg):
    pass

fun(1)
```

#### is equivalent to

```python
def fun(arg):
    pass
    
fun = decorator(fun)
 
fun(a)
```

### Implementaion
A decorator itself is a callable that return a callable

- Neseted function (Support both function and methods)
    - This is just a common pattern, not the only way

In [1]:
def decorator(fun):
    def wrapper(*args):
        print('[Args]: ', args)
        return fun(*args)
    return wrapper

# Function decorator
@decorator
def func(x, y):
    pass
    
func(6, 7)


class C:
    @decorator
    def method(self, x, y):
        pass

c = C()
c.method(1, 2)

[Args]:  (6, 7)
[Args]:  (<__main__.C object at 0x02D93410>, 1, 2)


## Class Decorators

### Usage

```python
@decorator
class C:
    pass

c = C(1)
```

#### is equivalent to

```python
class C:
    pass
    
C = decorator(C)
 
c = C(1)
```

### Implementation
Just like function decorators, though some may involve two levels of augmentation

In [2]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args):
            self.wrapped = cls(*args)
            self.calls = 0
            
        def __getattr__(self, name):
            self.calls += 1
            print('[name]: ', name)
            print('[calls]: ', self.calls)
            return getattr(self.wrapped, name)
    return Wrapper
    
@decorator
class C:
    def __init__(self, x, y):
        self.attr = 'spam'

c1 = C(6, 7)
print(c1.attr)

c2 = C(7, 8)
print(c2.attr)

[name]:  attr
[calls]:  1
spam
[name]:  attr
[calls]:  1
spam


- Supporting multiple instance
    - Handles multiple decorated classes (each makes a new Decorator instnace)

In [3]:
class Decorator:
    def __init__(self, C):                         # On @decoration
        self.C = C
        self.calls = 0
        
    def __call__(self, *args):                      # On instance creation
        self.calls += 1
        print('[Args]: ', args)
        print('[Calls]: ', self.calls)
        self.wrapped = self.C(*args)
        return self
    
    def __getattr__(self, attrname):
        return getattr(self.wrapped, attrname)
    
@Decorator
class C:
    def __init__(*args):
        pass

c1 = C(1)
c2 = C(2)

[Args]:  (1,)
[Calls]:  1
[Args]:  (2,)
[Calls]:  2


## Decorator Nesting

```python
@A
@B
@C
def fun(arg):
    pass
```

#### is equivalent to

```python
def fun(arg):
    pass
    
fun = A(B(C(fun)))
```

## Decorator Arguments
- Usually used to
    - retain state information for use in later calls
    - attribute initialization
    - atttribute names to be validated
- It often implied three levels of callables

### Usage
```python
@decorator(x, y)
def fun(arg):
    pass
    
fun(1)
```

#### is equivalent to

```python
def fun(arg):
    pass
    
fun = decorator(x, y)(fun)

fun(1)
```

### Implementation

```python
def decorator(x, y):
    # save or use x, y
    def actualDecorator(func):
        # save or use function func
        # return a callable: nested def, class with __call__ etc.
        return callable
    return actualDecorator
    
```